# MST

In [ ]:
import sys # Library for INT_MAX

class Graph:

  def __init__(self, graph, iCity):
    self.n = len(graph[0])
    self.graph = graph
    self.iCity = iCity
    #For MST:
    self.key = [sys.maxsize] * self.n # Key values used to pick minimum weight edge in cut
    self.parent = [None] * self.n # Array to store constructed MST
    self.mstSet = [False] * self.n
    ## Some initializations:
    self.key[iCity] = 0 		# Make key 0 so that this vertex is picked as first vertex
    self.parent[iCity] = -1 # First node is always the root of the tree
    self.totalMSTweight = 0

  def calcTotalWeight(self, parent):
    for i in range(self.n):
      if parent[i] != None and i!=self.iCity:
        self.totalMSTweight+=self.graph[i][parent[i]]

  def minKey(self):
    min = sys.maxsize 	# Initialize min value
    minIndex=-1
    for v in range(self.n):
      if self.key[v] < min and self.mstSet[v] == False:
        min = self.key[v]
        minIndex = v

    return minIndex

  def MST(self): #Prim's algorithm

    for _ in range(self.n):
      u = self.minKey() # min distance neighbour to the tree
      self.mstSet[u] = True #Record it

      for v in range(self.n): #Update its' neighbours' distances if needed
        if self.graph[u][v] > 0 and self.mstSet[v] == False and self.key[v] > self.graph[u][v]:
          self.key[v] = self.graph[u][v] # Update the key only if graph[u][v] is smaller than key[v]
          self.parent[v] = u

    self.calcTotalWeight(self.parent)
    return self.totalMSTweight
  

# Example driver code:
# graph = [[0,4,1,3],
#          [4,0,2,1],
#          [1,2,0,5],
#          [3,1,5,0]]
# g = Graph(graph)
# print(g.MST())

# A*

In [ ]:
class node: #Class to track the properties of each node in the search space
  def __init__(self, iCity, parent, depth, g, visSet):
    self.iCity = iCity 
    self.parent = parent # a node that generated the current node
    self.g = g # g(n) = cost from the parent state
    self.depth = depth #an integer
    self.visitedSet= visSet

In [ ]:
from queue import PriorityQueue
from copy import deepcopy
from itertools import count
# cities = {0: 'Zurich', 1:'B', 2:'C', 3:'D', 4: 'E'} 
cities = {0: 'Zurich',1: 'Warsaw',2: 'Vienna',3: 'Bucharest',4: 'Amsterdam',5: 'Venice',6: 'Stuttgart',7: 'Geneva'}
class tour: #class that tracks properties and search state space for a given graph configuration 
  def __init__(self, graph, init_node_iCity = 0):
    self.graph = Graph(graph, 0)
    initNode = node(init_node_iCity, None, 0, 0, set()) #initNode will be the root node
    self.currentNode=initNode # current Node that is being looked into
    self.fringeQ = PriorityQueue() #fringe list for generated nodes (but not expanded nodes)
    self.unique=count()
    self.fringeQ.put((0,next(self.unique),initNode)) 
    # self.visitedSet=set()#List for expanded states     
    self.unvGr = deepcopy(self.graph.graph)
  
  def tracePathFromRoot(self, aNode: node): #when we get a solution we print the path, recursively. Prints from parent node
    if aNode == None:
      return
    self.tracePathFromRoot(aNode.parent)
    print('At Level: ', aNode.depth, ", go to: ", cities[aNode.iCity], ". Total cost of tour till now: ", aNode.g)
  
  def childGen(self, parentNode: node, iCityChild, wt, h): #Generating the child node given the move
    g = parentNode.g + wt
    fCost = g+h #f(n) = g(n) + h(n)
    cVisSet = parentNode.visitedSet.copy()
    child = node(iCityChild, parentNode, parentNode.depth+1, g, cVisSet)
    self.fringeQ.put((fCost,next(self.unique),child))
   
  def goal(self):
    if self.currentNode.depth+1 ==self.graph.n and len(self.currentNode.visitedSet)==self.graph.n:
      return True
    else:
      return False

In [ ]:
def successorFn(tsp: tour):
  status = ' '
  while not tsp.fringeQ.empty(): 
    tsp.currentNode = tsp.fringeQ.get()[2] #dequeue to get current parent node
    #finally, as all the moves from the current node will be enumerated, archive the node
    tsp.currentNode.visitedSet.add(tsp.currentNode.iCity)    
    if tsp.goal(): #Stop search
      tsp.tracePathFromRoot(tsp.currentNode) #just stop all node searching/generation and print the path
      backStart = tsp.graph.graph[0][tsp.currentNode.iCity]
      print("Going back to Hometown Zurich from the last stop ", cities[tsp.currentNode.iCity], " costs: ", backStart)
      print("Total Cost of Tour: ", backStart+tsp.currentNode.g)
      status = 'Solved'
      return status
    else: #Generate further
      #Calculate the MST of the graph without the current dequeued node. to get the h for the next nodes that will be generated
      tsp.unvGr = deepcopy(tsp.graph.graph) 
      for i in range(tsp.graph.n): 
        for j in tsp.currentNode.visitedSet:
          tsp.unvGr[j][i] = sys.maxsize
          tsp.unvGr[i][j] = sys.maxsize

      #actual node generation by going through the neighbours
      for iCityChild, wt in enumerate(tsp.graph.graph[tsp.currentNode.iCity]):
        if wt!=0 and iCityChild not in tsp.currentNode.visitedSet:
          unvisitedGraph = Graph(tsp.unvGr, iCityChild) #could have done outside the loop but need to specify a starting point anyhow
          h = unvisitedGraph.MST()
          tsp.childGen(tsp.currentNode, iCityChild, wt, h)

  if(status!='Solved'): #if fringe list is empty, it means all possible states were already explored, so no solution exists. Happens when the graph given is not connected suitably
    status = 'Unsolvable'
  
  return status

# Driver Code

In [ ]:
#Driver code
# graph = [[0,15,13,16, 12],
#          [15,0,14,11,17],
#          [13,14,0,19,18],
#          [16,11,19,0, 14],
#          [12, 17, 18, 14, 0]]
# Random symmetric matrix with 0 diagonals generated from: https://catonmat.net/tools/generate-symmetric-matrices
# x = '0,9,18,5,18,13,19,10,4,15,17,20,6,20,1,17,4,14,9,10,15,7,18,8,6,9,12,17,19,13,20,5,18,8,18,2,1,17,9,12,16,20,12,8,10,6,10,11,11,14/9,0,8,15,7,4,20,8,18,15,14,14,9,3,3,19,11,5,11,15,6,14,6,16,11,17,1,14,14,11,16,4,7,3,1,15,10,9,10,3,16,8,10,1,8,9,17,8,8,1/18,8,0,18,8,2,17,4,10,17,9,14,20,16,6,15,5,10,11,7,12,10,5,13,7,9,11,16,10,15,8,11,9,15,16,13,9,6,5,16,17,16,8,11,8,6,12,8,12,7/5,15,18,0,4,12,9,10,15,17,16,18,11,3,16,1,12,18,10,15,14,8,15,12,5,5,20,2,20,9,7,7,10,19,6,5,1,2,18,18,3,2,17,3,15,20,17,7,14,2/18,7,8,4,0,4,12,9,3,4,2,5,8,9,16,20,7,12,19,10,3,16,18,8,5,11,2,16,12,17,14,3,8,13,10,8,17,1,11,14,3,2,10,18,1,8,14,18,19,10/13,4,2,12,4,0,18,3,6,18,8,7,5,12,9,1,5,1,17,19,3,17,15,1,4,4,18,9,13,6,15,15,7,15,9,2,18,13,17,14,18,18,8,19,3,2,17,4,13,20/19,20,17,9,12,18,0,3,7,14,20,3,14,18,19,16,18,17,9,9,8,5,14,4,12,16,10,16,6,10,14,17,4,12,6,11,11,5,13,4,9,12,15,6,6,12,4,9,18,5/10,8,4,10,9,3,3,0,12,8,2,19,7,9,8,16,1,15,2,6,1,10,12,6,17,1,10,14,1,20,7,13,12,6,9,7,3,8,8,15,11,16,12,12,9,14,5,14,17,15/4,18,10,15,3,6,7,12,0,12,4,18,16,12,12,16,19,20,5,11,11,17,20,10,13,7,15,18,7,7,5,11,2,17,20,3,15,5,7,20,16,11,2,13,20,14,2,19,10,1/15,15,17,17,4,18,14,8,12,0,11,10,20,9,2,19,17,19,10,11,9,10,6,11,18,12,7,2,11,5,4,2,18,9,9,14,4,17,20,17,18,6,4,4,15,4,5,16,17,7/17,14,9,16,2,8,20,2,4,11,0,18,5,13,8,14,12,20,18,4,12,6,9,16,11,17,18,9,14,9,11,1,14,9,5,18,15,19,4,5,16,16,11,17,8,13,6,1,2,4/20,14,14,18,5,7,3,19,18,10,18,0,20,16,17,16,2,14,3,14,2,13,1,18,18,2,14,10,16,11,13,1,16,11,1,5,3,10,2,2,19,14,3,12,16,5,12,19,4,4/6,9,20,11,8,5,14,7,16,20,5,20,0,17,16,5,6,16,15,16,9,13,2,12,11,5,13,17,6,16,12,13,12,16,6,12,2,5,1,20,16,12,4,17,16,17,11,10,15,11/20,3,16,3,9,12,18,9,12,9,13,16,17,0,1,12,19,11,5,2,6,20,9,3,20,15,3,17,13,1,16,3,14,5,1,14,10,16,1,10,1,6,15,15,20,2,1,17,6,16/1,3,6,16,16,9,19,8,12,2,8,17,16,1,0,17,7,12,13,1,6,15,10,16,7,12,2,15,8,3,10,13,15,10,3,9,5,20,10,7,7,20,2,14,2,2,7,7,7,20/17,19,15,1,20,1,16,16,16,19,14,16,5,12,17,0,13,10,16,4,20,12,18,8,12,1,8,10,20,10,11,7,10,16,8,19,10,8,1,13,12,5,14,9,14,1,10,15,12,8/4,11,5,12,7,5,18,1,19,17,12,2,6,19,7,13,0,13,10,2,18,6,4,6,20,15,2,1,6,20,14,16,11,7,3,8,1,2,5,5,9,18,10,19,5,15,5,17,2,18/14,5,10,18,12,1,17,15,20,19,20,14,16,11,12,10,13,0,18,17,2,17,16,14,3,3,16,5,10,18,13,2,9,17,4,4,4,7,6,18,2,20,5,18,19,18,3,1,3,6/9,11,11,10,19,17,9,2,5,10,18,3,15,5,13,16,10,18,0,14,11,14,2,16,15,6,12,4,7,16,12,15,19,11,2,19,15,5,5,6,10,7,10,18,4,5,11,7,16,10/10,15,7,15,10,19,9,6,11,11,4,14,16,2,1,4,2,17,14,0,7,9,19,18,5,11,4,7,20,16,4,8,7,17,19,4,10,15,14,13,16,14,12,2,15,14,4,4,16,5/15,6,12,14,3,3,8,1,11,9,12,2,9,6,6,20,18,2,11,7,0,18,1,7,9,17,1,16,1,16,5,8,16,9,15,12,7,3,10,15,5,2,10,11,2,12,17,3,8,1/7,14,10,8,16,17,5,10,17,10,6,13,13,20,15,12,6,17,14,9,18,0,13,5,6,2,8,3,5,2,5,4,9,15,8,14,17,15,19,13,9,14,20,2,12,9,7,9,5,6/18,6,5,15,18,15,14,12,20,6,9,1,2,9,10,18,4,16,2,19,1,13,0,18,6,3,15,20,15,2,16,17,14,4,19,14,20,4,17,14,10,7,11,9,12,2,13,7,13,12/8,16,13,12,8,1,4,6,10,11,16,18,12,3,16,8,6,14,16,18,7,5,18,0,1,9,18,2,9,11,17,3,1,2,6,18,2,17,5,9,10,15,15,9,2,8,7,3,15,18/6,11,7,5,5,4,12,17,13,18,11,18,11,20,7,12,20,3,15,5,9,6,6,1,0,9,10,5,4,9,16,2,19,14,13,10,17,5,10,16,17,8,9,14,12,9,17,3,14,2/9,17,9,5,11,4,16,1,7,12,17,2,5,15,12,1,15,3,6,11,17,2,3,9,9,0,12,8,7,14,9,15,7,5,17,5,8,16,6,9,3,5,8,12,16,5,4,13,8,19/12,1,11,20,2,18,10,10,15,7,18,14,13,3,2,8,2,16,12,4,1,8,15,18,10,12,0,7,7,7,13,15,19,3,13,5,2,7,18,20,6,7,20,10,16,14,6,16,17,1/17,14,16,2,16,9,16,14,18,2,9,10,17,17,15,10,1,5,4,7,16,3,20,2,5,8,7,0,8,5,10,20,7,4,3,3,20,4,7,1,1,13,2,18,10,13,20,15,13,2/19,14,10,20,12,13,6,1,7,11,14,16,6,13,8,20,6,10,7,20,1,5,15,9,4,7,7,8,0,5,15,18,10,15,14,10,7,16,13,8,12,9,2,14,7,14,3,11,15,5/13,11,15,9,17,6,10,20,7,5,9,11,16,1,3,10,20,18,16,16,16,2,2,11,9,14,7,5,5,0,11,19,2,16,7,16,11,9,18,14,5,7,3,1,4,10,14,7,12,4/20,16,8,7,14,15,14,7,5,4,11,13,12,16,10,11,14,13,12,4,5,5,16,17,16,9,13,10,15,11,0,19,6,17,15,12,9,10,12,16,5,8,3,19,18,12,8,4,18,10/5,4,11,7,3,15,17,13,11,2,1,1,13,3,13,7,16,2,15,8,8,4,17,3,2,15,15,20,18,19,19,0,19,1,15,9,3,18,13,6,16,1,8,19,20,20,10,1,12,2/18,7,9,10,8,7,4,12,2,18,14,16,12,14,15,10,11,9,19,7,16,9,14,1,19,7,19,7,10,2,6,19,0,17,12,3,3,4,5,7,20,20,7,16,5,2,19,9,1,13/8,3,15,19,13,15,12,6,17,9,9,11,16,5,10,16,7,17,11,17,9,15,4,2,14,5,3,4,15,16,17,1,17,0,7,3,17,5,13,10,7,9,17,3,18,2,3,3,14,20/18,1,16,6,10,9,6,9,20,9,5,1,6,1,3,8,3,4,2,19,15,8,19,6,13,17,13,3,14,7,15,15,12,7,0,12,5,17,8,3,14,12,8,1,17,12,13,1,8,6/2,15,13,5,8,2,11,7,3,14,18,5,12,14,9,19,8,4,19,4,12,14,14,18,10,5,5,3,10,16,12,9,3,3,12,0,9,20,10,9,13,18,11,4,16,11,5,18,3,4/1,10,9,1,17,18,11,3,15,4,15,3,2,10,5,10,1,4,15,10,7,17,20,2,17,8,2,20,7,11,9,3,3,17,5,9,0,3,16,11,6,6,6,20,15,6,4,16,19,16/17,9,6,2,1,13,5,8,5,17,19,10,5,16,20,8,2,7,5,15,3,15,4,17,5,16,7,4,16,9,10,18,4,5,17,20,3,0,13,16,20,13,16,3,17,10,9,15,12,19/9,10,5,18,11,17,13,8,7,20,4,2,1,1,10,1,5,6,5,14,10,19,17,5,10,6,18,7,13,18,12,13,5,13,8,10,16,13,0,8,4,12,8,9,5,14,15,6,8,2/12,3,16,18,14,14,4,15,20,17,5,2,20,10,7,13,5,18,6,13,15,13,14,9,16,9,20,1,8,14,16,6,7,10,3,9,11,16,8,0,8,16,20,5,4,10,9,17,20,12/16,16,17,3,3,18,9,11,16,18,16,19,16,1,7,12,9,2,10,16,5,9,10,10,17,3,6,1,12,5,5,16,20,7,14,13,6,20,4,8,0,5,17,6,3,12,17,2,7,18/20,8,16,2,2,18,12,16,11,6,16,14,12,6,20,5,18,20,7,14,2,14,7,15,8,5,7,13,9,7,8,1,20,9,12,18,6,13,12,16,5,0,16,2,2,12,6,2,17,12/12,10,8,17,10,8,15,12,2,4,11,3,4,15,2,14,10,5,10,12,10,20,11,15,9,8,20,2,2,3,3,8,7,17,8,11,6,16,8,20,17,16,0,9,7,17,13,16,14,6/8,1,11,3,18,19,6,12,13,4,17,12,17,15,14,9,19,18,18,2,11,2,9,9,14,12,10,18,14,1,19,19,16,3,1,4,20,3,9,5,6,2,9,0,1,9,7,13,17,6/10,8,8,15,1,3,6,9,20,15,8,16,16,20,2,14,5,19,4,15,2,12,12,2,12,16,16,10,7,4,18,20,5,18,17,16,15,17,5,4,3,2,7,1,0,5,6,20,6,7/6,9,6,20,8,2,12,14,14,4,13,5,17,2,2,1,15,18,5,14,12,9,2,8,9,5,14,13,14,10,12,20,2,2,12,11,6,10,14,10,12,12,17,9,5,0,2,9,2,17/10,17,12,17,14,17,4,5,2,5,6,12,11,1,7,10,5,3,11,4,17,7,13,7,17,4,6,20,3,14,8,10,19,3,13,5,4,9,15,9,17,6,13,7,6,2,0,11,3,2/11,8,8,7,18,4,9,14,19,16,1,19,10,17,7,15,17,1,7,4,3,9,7,3,3,13,16,15,11,7,4,1,9,3,1,18,16,15,6,17,2,2,16,13,20,9,11,0,9,8/11,8,12,14,19,13,18,17,10,17,2,4,15,6,7,12,2,3,16,16,8,5,13,15,14,8,17,13,15,12,18,12,1,14,8,3,19,12,8,20,7,17,14,17,6,2,3,9,0,5/14,1,7,2,10,20,5,15,1,7,4,4,11,16,20,8,18,6,10,5,1,6,12,18,2,19,1,2,5,4,10,2,13,20,6,4,16,19,2,12,18,12,6,6,7,17,2,8,5,0'
# x = '0 ,2 ,4 ,4 ,0 ,0 ,5 ,4 ,6 ,8 ,1 ,8 ,1 ,0 ,0 ,1 ,4 ,5 ,0 ,3 ,0 ,9 ,9 ,1 ,6 ,1 ,3 ,0 ,5 ,8 ,2 ,9 ,1 ,5 ,5 ,1 ,6 ,0 ,6 ,4 ,4 ,0 ,5 ,1 ,7 ,2 ,6 ,9 ,8 ,2/2 ,0 ,1 ,7 ,9 ,9 ,8 ,6 ,6 ,2 ,3 ,6 ,6 ,7 ,6 ,9 ,3 ,6 ,1 ,0 ,2 ,3 ,6 ,6 ,7 ,2 ,1 ,3 ,8 ,7 ,9 ,3 ,4 ,1 ,2 ,0 ,1 ,6 ,0 ,6 ,5 ,3 ,0 ,7 ,8 ,9 ,1 ,2 ,1 ,7/4 ,1 ,0 ,2 ,0 ,3 ,2 ,8 ,8 ,6 ,8 ,9 ,0 ,9 ,0 ,6 ,0 ,4 ,9 ,5 ,4 ,5 ,3 ,2 ,5 ,7 ,2 ,1 ,4 ,0 ,7 ,8 ,1 ,7 ,6 ,4 ,9 ,8 ,1 ,3 ,3 ,3 ,7 ,3 ,2 ,0 ,2 ,3 ,0 ,0/4 ,7 ,2 ,0 ,2 ,7 ,0 ,7 ,4 ,6 ,3 ,6 ,4 ,6 ,5 ,2 ,1 ,5 ,0 ,8 ,4 ,4 ,8 ,7 ,2 ,7 ,8 ,3 ,0 ,9 ,8 ,6 ,2 ,8 ,6 ,9 ,7 ,8 ,7 ,5 ,9 ,6 ,5 ,9 ,5 ,1 ,2 ,4 ,8 ,0/0 ,9 ,0 ,2 ,0 ,6 ,2 ,6 ,6 ,4 ,6 ,6 ,8 ,2 ,2 ,7 ,1 ,3 ,5 ,0 ,9 ,9 ,1 ,3 ,0 ,0 ,4 ,0 ,3 ,3 ,5 ,3 ,8 ,4 ,7 ,4 ,0 ,2 ,4 ,9 ,1 ,1 ,5 ,5 ,9 ,9 ,4 ,6 ,3 ,2/0 ,9 ,3 ,7 ,6 ,0 ,6 ,5 ,9 ,2 ,8 ,2 ,7 ,5 ,1 ,0 ,7 ,9 ,5 ,3 ,9 ,4 ,5 ,9 ,7 ,2 ,5 ,3 ,2 ,4 ,7 ,9 ,1 ,9 ,3 ,7 ,0 ,7 ,4 ,4 ,6 ,3 ,5 ,0 ,5 ,9 ,7 ,9 ,2 ,7/5 ,8 ,2 ,0 ,2 ,6 ,0 ,2 ,8 ,1 ,1 ,6 ,8 ,0 ,7 ,6 ,4 ,8 ,5 ,1 ,1 ,4 ,8 ,7 ,2 ,3 ,9 ,2 ,2 ,8 ,5 ,2 ,3 ,4 ,0 ,5 ,7 ,3 ,0 ,4 ,3 ,5 ,3 ,5 ,4 ,6 ,1 ,4 ,1 ,9/4 ,6 ,8 ,7 ,6 ,5 ,2 ,0 ,7 ,7 ,8 ,0 ,5 ,3 ,1 ,8 ,7 ,0 ,4 ,6 ,8 ,7 ,1 ,1 ,6 ,5 ,9 ,3 ,2 ,5 ,6 ,9 ,0 ,3 ,4 ,1 ,6 ,6 ,3 ,5 ,1 ,2 ,3 ,1 ,1 ,0 ,4 ,4 ,6 ,3/6 ,6 ,8 ,4 ,6 ,9 ,8 ,7 ,0 ,0 ,9 ,8 ,8 ,8 ,0 ,6 ,4 ,5 ,2 ,9 ,8 ,4 ,0 ,9 ,0 ,8 ,2 ,1 ,8 ,9 ,2 ,9 ,7 ,5 ,9 ,5 ,2 ,8 ,7 ,2 ,5 ,7 ,1 ,1 ,5 ,1 ,7 ,6 ,8 ,3/8 ,2 ,6 ,6 ,4 ,2 ,1 ,7 ,0 ,0 ,3 ,5 ,2 ,3 ,8 ,1 ,3 ,0 ,9 ,8 ,2 ,5 ,7 ,6 ,7 ,5 ,8 ,3 ,4 ,0 ,3 ,1 ,4 ,5 ,1 ,0 ,8 ,7 ,3 ,9 ,9 ,1 ,6 ,4 ,2 ,7 ,3 ,0 ,2 ,9/1 ,3 ,8 ,3 ,6 ,8 ,1 ,8 ,9 ,3 ,0 ,0 ,7 ,1 ,4 ,4 ,9 ,5 ,9 ,6 ,9 ,8 ,7 ,4 ,1 ,4 ,3 ,5 ,0 ,2 ,1 ,1 ,3 ,9 ,0 ,6 ,0 ,7 ,8 ,6 ,0 ,0 ,1 ,0 ,4 ,1 ,6 ,8 ,4 ,2/8 ,6 ,9 ,6 ,6 ,2 ,6 ,0 ,8 ,5 ,0 ,0 ,1 ,0 ,6 ,1 ,1 ,4 ,8 ,9 ,5 ,5 ,8 ,2 ,4 ,0 ,9 ,9 ,7 ,9 ,4 ,7 ,5 ,9 ,2 ,5 ,7 ,5 ,0 ,3 ,6 ,6 ,9 ,3 ,5 ,7 ,8 ,7 ,0 ,3/1 ,6 ,0 ,4 ,8 ,7 ,8 ,5 ,8 ,2 ,7 ,1 ,0 ,2 ,2 ,0 ,0 ,0 ,7 ,9 ,2 ,6 ,3 ,4 ,1 ,8 ,8 ,0 ,7 ,0 ,8 ,8 ,3 ,5 ,2 ,2 ,7 ,1 ,0 ,6 ,9 ,1 ,3 ,6 ,4 ,5 ,5 ,0 ,7 ,6/0 ,7 ,9 ,6 ,2 ,5 ,0 ,3 ,8 ,3 ,1 ,0 ,2 ,0 ,3 ,4 ,6 ,5 ,6 ,9 ,5 ,5 ,1 ,9 ,0 ,4 ,0 ,6 ,0 ,6 ,5 ,2 ,1 ,0 ,9 ,6 ,4 ,4 ,8 ,7 ,5 ,3 ,3 ,2 ,3 ,1 ,1 ,1 ,5 ,0/0 ,6 ,0 ,5 ,2 ,1 ,7 ,1 ,0 ,8 ,4 ,6 ,2 ,3 ,0 ,8 ,8 ,1 ,3 ,7 ,4 ,4 ,8 ,0 ,7 ,6 ,9 ,9 ,7 ,2 ,1 ,7 ,0 ,5 ,1 ,1 ,6 ,7 ,3 ,6 ,3 ,4 ,6 ,8 ,0 ,5 ,7 ,9 ,5 ,8/1 ,9 ,6 ,2 ,7 ,0 ,6 ,8 ,6 ,1 ,4 ,1 ,0 ,4 ,8 ,0 ,5 ,7 ,9 ,8 ,6 ,5 ,7 ,4 ,0 ,3 ,0 ,1 ,1 ,2 ,6 ,6 ,2 ,6 ,6 ,8 ,2 ,4 ,7 ,3 ,9 ,5 ,7 ,5 ,6 ,5 ,7 ,3 ,7 ,0/4 ,3 ,0 ,1 ,1 ,7 ,4 ,7 ,4 ,3 ,9 ,1 ,0 ,6 ,8 ,5 ,0 ,1 ,4 ,0 ,3 ,2 ,0 ,6 ,5 ,0 ,1 ,8 ,6 ,9 ,5 ,3 ,2 ,1 ,5 ,6 ,3 ,2 ,3 ,8 ,8 ,3 ,0 ,0 ,2 ,6 ,3 ,7 ,7 ,2/5 ,6 ,4 ,5 ,3 ,9 ,8 ,0 ,5 ,0 ,5 ,4 ,0 ,5 ,1 ,7 ,1 ,0 ,3 ,4 ,0 ,1 ,5 ,0 ,1 ,9 ,8 ,4 ,9 ,7 ,5 ,0 ,4 ,7 ,8 ,2 ,8 ,4 ,9 ,8 ,5 ,5 ,8 ,1 ,4 ,1 ,7 ,2 ,6 ,3/0 ,1 ,9 ,0 ,5 ,5 ,5 ,4 ,2 ,9 ,9 ,8 ,7 ,6 ,3 ,9 ,4 ,3 ,0 ,4 ,7 ,1 ,5 ,0 ,4 ,9 ,8 ,9 ,8 ,1 ,6 ,1 ,0 ,6 ,1 ,2 ,1 ,4 ,3 ,7 ,1 ,1 ,4 ,5 ,0 ,3 ,8 ,8 ,9 ,4/3 ,0 ,5 ,8 ,0 ,3 ,1 ,6 ,9 ,8 ,6 ,9 ,9 ,9 ,7 ,8 ,0 ,4 ,4 ,0 ,5 ,2 ,8 ,2 ,2 ,7 ,8 ,5 ,5 ,5 ,7 ,6 ,3 ,5 ,4 ,9 ,7 ,0 ,2 ,7 ,3 ,2 ,5 ,1 ,3 ,8 ,2 ,6 ,0 ,2/0 ,2 ,4 ,4 ,9 ,9 ,1 ,8 ,8 ,2 ,9 ,5 ,2 ,5 ,4 ,6 ,3 ,0 ,7 ,5 ,0 ,4 ,9 ,6 ,7 ,5 ,0 ,6 ,1 ,4 ,6 ,7 ,8 ,5 ,1 ,7 ,1 ,9 ,7 ,3 ,3 ,3 ,9 ,5 ,7 ,9 ,2 ,1 ,3 ,6/9 ,3 ,5 ,4 ,9 ,4 ,4 ,7 ,4 ,5 ,8 ,5 ,6 ,5 ,4 ,5 ,2 ,1 ,1 ,2 ,4 ,0 ,8 ,0 ,3 ,8 ,2 ,4 ,7 ,5 ,1 ,9 ,5 ,4 ,8 ,1 ,9 ,0 ,1 ,8 ,3 ,8 ,3 ,9 ,4 ,4 ,7 ,8 ,6 ,4/9 ,6 ,3 ,8 ,1 ,5 ,8 ,1 ,0 ,7 ,7 ,8 ,3 ,1 ,8 ,7 ,0 ,5 ,5 ,8 ,9 ,8 ,0 ,0 ,7 ,4 ,9 ,6 ,0 ,5 ,6 ,8 ,6 ,9 ,8 ,6 ,0 ,3 ,2 ,7 ,9 ,5 ,8 ,0 ,8 ,3 ,8 ,0 ,7 ,4/1 ,6 ,2 ,7 ,3 ,9 ,7 ,1 ,9 ,6 ,4 ,2 ,4 ,9 ,0 ,4 ,6 ,0 ,0 ,2 ,6 ,0 ,0 ,0 ,7 ,1 ,0 ,8 ,0 ,2 ,6 ,2 ,3 ,0 ,0 ,3 ,6 ,9 ,5 ,0 ,0 ,1 ,9 ,0 ,7 ,0 ,3 ,3 ,3 ,1/6 ,7 ,5 ,2 ,0 ,7 ,2 ,6 ,0 ,7 ,1 ,4 ,1 ,0 ,7 ,0 ,5 ,1 ,4 ,2 ,7 ,3 ,7 ,7 ,0 ,6 ,1 ,0 ,7 ,1 ,7 ,6 ,4 ,3 ,0 ,4 ,5 ,5 ,1 ,9 ,3 ,4 ,4 ,6 ,3 ,0 ,5 ,7 ,9 ,1/1 ,2 ,7 ,7 ,0 ,2 ,3 ,5 ,8 ,5 ,4 ,0 ,8 ,4 ,6 ,3 ,0 ,9 ,9 ,7 ,5 ,8 ,4 ,1 ,6 ,0 ,6 ,2 ,6 ,6 ,3 ,8 ,8 ,9 ,1 ,3 ,3 ,1 ,0 ,1 ,2 ,7 ,2 ,6 ,9 ,7 ,3 ,8 ,4 ,5/3 ,1 ,2 ,8 ,4 ,5 ,9 ,9 ,2 ,8 ,3 ,9 ,8 ,0 ,9 ,0 ,1 ,8 ,8 ,8 ,0 ,2 ,9 ,0 ,1 ,6 ,0 ,2 ,5 ,9 ,0 ,4 ,8 ,4 ,4 ,7 ,8 ,5 ,2 ,2 ,1 ,2 ,1 ,5 ,3 ,1 ,0 ,4 ,4 ,0/0 ,3 ,1 ,3 ,0 ,3 ,2 ,3 ,1 ,3 ,5 ,9 ,0 ,6 ,9 ,1 ,8 ,4 ,9 ,5 ,6 ,4 ,6 ,8 ,0 ,2 ,2 ,0 ,8 ,6 ,4 ,2 ,0 ,6 ,3 ,2 ,2 ,6 ,3 ,5 ,0 ,9 ,6 ,7 ,5 ,4 ,4 ,8 ,2 ,0/5 ,8 ,4 ,0 ,3 ,2 ,2 ,2 ,8 ,4 ,0 ,7 ,7 ,0 ,7 ,1 ,6 ,9 ,8 ,5 ,1 ,7 ,0 ,0 ,7 ,6 ,5 ,8 ,0 ,6 ,2 ,3 ,1 ,4 ,4 ,5 ,8 ,4 ,4 ,2 ,3 ,2 ,0 ,0 ,1 ,4 ,0 ,6 ,0 ,6/8 ,7 ,0 ,9 ,3 ,4 ,8 ,5 ,9 ,0 ,2 ,9 ,0 ,6 ,2 ,2 ,9 ,7 ,1 ,5 ,4 ,5 ,5 ,2 ,1 ,6 ,9 ,6 ,6 ,0 ,2 ,8 ,3 ,3 ,1 ,8 ,3 ,5 ,3 ,2 ,2 ,9 ,6 ,2 ,0 ,4 ,7 ,9 ,7 ,0/2 ,9 ,7 ,8 ,5 ,7 ,5 ,6 ,2 ,3 ,1 ,4 ,8 ,5 ,1 ,6 ,5 ,5 ,6 ,7 ,6 ,1 ,6 ,6 ,7 ,3 ,0 ,4 ,2 ,2 ,0 ,2 ,1 ,0 ,6 ,0 ,6 ,6 ,1 ,1 ,4 ,7 ,0 ,4 ,3 ,5 ,8 ,1 ,5 ,9/9 ,3 ,8 ,6 ,3 ,9 ,2 ,9 ,9 ,1 ,1 ,7 ,8 ,2 ,7 ,6 ,3 ,0 ,1 ,6 ,7 ,9 ,8 ,2 ,6 ,8 ,4 ,2 ,3 ,8 ,2 ,0 ,7 ,0 ,2 ,7 ,9 ,9 ,6 ,5 ,9 ,7 ,0 ,7 ,0 ,3 ,5 ,1 ,8 ,3/1 ,4 ,1 ,2 ,8 ,1 ,3 ,0 ,7 ,4 ,3 ,5 ,3 ,1 ,0 ,2 ,2 ,4 ,0 ,3 ,8 ,5 ,6 ,3 ,4 ,8 ,8 ,0 ,1 ,3 ,1 ,7 ,0 ,7 ,8 ,5 ,2 ,0 ,4 ,2 ,3 ,3 ,9 ,5 ,2 ,8 ,5 ,0 ,1 ,5/5 ,1 ,7 ,8 ,4 ,9 ,4 ,3 ,5 ,5 ,9 ,9 ,5 ,0 ,5 ,6 ,1 ,7 ,6 ,5 ,5 ,4 ,9 ,0 ,3 ,9 ,4 ,6 ,4 ,3 ,0 ,0 ,7 ,0 ,7 ,0 ,1 ,6 ,2 ,5 ,1 ,6 ,3 ,7 ,0 ,1 ,3 ,9 ,2 ,0/5 ,2 ,6 ,6 ,7 ,3 ,0 ,4 ,9 ,1 ,0 ,2 ,2 ,9 ,1 ,6 ,5 ,8 ,1 ,4 ,1 ,8 ,8 ,0 ,0 ,1 ,4 ,3 ,4 ,1 ,6 ,2 ,8 ,7 ,0 ,4 ,9 ,5 ,2 ,2 ,2 ,7 ,7 ,9 ,4 ,6 ,4 ,8 ,4 ,4/1 ,0 ,4 ,9 ,4 ,7 ,5 ,1 ,5 ,0 ,6 ,5 ,2 ,6 ,1 ,8 ,6 ,2 ,2 ,9 ,7 ,1 ,6 ,3 ,4 ,3 ,7 ,2 ,5 ,8 ,0 ,7 ,5 ,0 ,4 ,0 ,7 ,7 ,5 ,6 ,5 ,9 ,9 ,5 ,7 ,6 ,5 ,7 ,4 ,3/6 ,1 ,9 ,7 ,0 ,0 ,7 ,6 ,2 ,8 ,0 ,7 ,7 ,4 ,6 ,2 ,3 ,8 ,1 ,7 ,1 ,9 ,0 ,6 ,5 ,3 ,8 ,2 ,8 ,3 ,6 ,9 ,2 ,1 ,9 ,7 ,0 ,0 ,9 ,3 ,1 ,5 ,7 ,4 ,8 ,6 ,8 ,6 ,3 ,9/0 ,6 ,8 ,8 ,2 ,7 ,3 ,6 ,8 ,7 ,7 ,5 ,1 ,4 ,7 ,4 ,2 ,4 ,4 ,0 ,9 ,0 ,3 ,9 ,5 ,1 ,5 ,6 ,4 ,5 ,6 ,9 ,0 ,6 ,5 ,7 ,0 ,0 ,5 ,8 ,7 ,5 ,1 ,6 ,5 ,8 ,6 ,3 ,2 ,8/6 ,0 ,1 ,7 ,4 ,4 ,0 ,3 ,7 ,3 ,8 ,0 ,0 ,8 ,3 ,7 ,3 ,9 ,3 ,2 ,7 ,1 ,2 ,5 ,1 ,0 ,2 ,3 ,4 ,3 ,1 ,6 ,4 ,2 ,2 ,5 ,9 ,5 ,0 ,3 ,6 ,3 ,3 ,6 ,0 ,9 ,8 ,1 ,4 ,6/4 ,6 ,3 ,5 ,9 ,4 ,4 ,5 ,2 ,9 ,6 ,3 ,6 ,7 ,6 ,3 ,8 ,8 ,7 ,7 ,3 ,8 ,7 ,0 ,9 ,1 ,2 ,5 ,2 ,2 ,1 ,5 ,2 ,5 ,2 ,6 ,3 ,8 ,3 ,0 ,0 ,0 ,2 ,7 ,2 ,4 ,4 ,6 ,6 ,8/4 ,5 ,3 ,9 ,1 ,6 ,3 ,1 ,5 ,9 ,0 ,6 ,9 ,5 ,3 ,9 ,8 ,5 ,1 ,3 ,3 ,3 ,9 ,0 ,3 ,2 ,1 ,0 ,3 ,2 ,4 ,9 ,3 ,1 ,2 ,5 ,1 ,7 ,6 ,0 ,0 ,0 ,1 ,6 ,2 ,8 ,2 ,5 ,7 ,3/0 ,3 ,3 ,6 ,1 ,3 ,5 ,2 ,7 ,1 ,0 ,6 ,1 ,3 ,4 ,5 ,3 ,5 ,1 ,2 ,3 ,8 ,5 ,1 ,4 ,7 ,2 ,9 ,2 ,9 ,7 ,7 ,3 ,6 ,7 ,9 ,5 ,5 ,3 ,0 ,0 ,0 ,8 ,0 ,3 ,2 ,8 ,0 ,6 ,8/5 ,0 ,7 ,5 ,5 ,5 ,3 ,3 ,1 ,6 ,1 ,9 ,3 ,3 ,6 ,7 ,0 ,8 ,4 ,5 ,9 ,3 ,8 ,9 ,4 ,2 ,1 ,6 ,0 ,6 ,0 ,0 ,9 ,3 ,7 ,9 ,7 ,1 ,3 ,2 ,1 ,8 ,0 ,0 ,8 ,8 ,3 ,8 ,7 ,1/1 ,7 ,3 ,9 ,5 ,0 ,5 ,1 ,1 ,4 ,0 ,3 ,6 ,2 ,8 ,5 ,0 ,1 ,5 ,1 ,5 ,9 ,0 ,0 ,6 ,6 ,5 ,7 ,0 ,2 ,4 ,7 ,5 ,7 ,9 ,5 ,4 ,6 ,6 ,7 ,6 ,0 ,0 ,0 ,8 ,6 ,4 ,0 ,5 ,2/7 ,8 ,2 ,5 ,9 ,5 ,4 ,1 ,5 ,2 ,4 ,5 ,4 ,3 ,0 ,6 ,2 ,4 ,0 ,3 ,7 ,4 ,8 ,7 ,3 ,9 ,3 ,5 ,1 ,0 ,3 ,0 ,2 ,0 ,4 ,7 ,8 ,5 ,0 ,2 ,2 ,3 ,8 ,8 ,0 ,5 ,9 ,2 ,9 ,5/2 ,9 ,0 ,1 ,9 ,9 ,6 ,0 ,1 ,7 ,1 ,7 ,5 ,1 ,5 ,5 ,6 ,1 ,3 ,8 ,9 ,4 ,3 ,0 ,0 ,7 ,1 ,4 ,4 ,4 ,5 ,3 ,8 ,1 ,6 ,6 ,6 ,8 ,9 ,4 ,8 ,2 ,8 ,6 ,5 ,0 ,5 ,7 ,2 ,2/6 ,1 ,2 ,2 ,4 ,7 ,1 ,4 ,7 ,3 ,6 ,8 ,5 ,1 ,7 ,7 ,3 ,7 ,8 ,2 ,2 ,7 ,8 ,3 ,5 ,3 ,0 ,4 ,0 ,7 ,8 ,5 ,5 ,3 ,4 ,5 ,8 ,6 ,8 ,4 ,2 ,8 ,3 ,4 ,9 ,5 ,0 ,8 ,0 ,8/9 ,2 ,3 ,4 ,6 ,9 ,4 ,4 ,6 ,0 ,8 ,7 ,0 ,1 ,9 ,3 ,7 ,2 ,8 ,6 ,1 ,8 ,0 ,3 ,7 ,8 ,4 ,8 ,6 ,9 ,1 ,1 ,0 ,9 ,8 ,7 ,6 ,3 ,1 ,6 ,5 ,0 ,8 ,0 ,2 ,7 ,8 ,0 ,1 ,9/8 ,1 ,0 ,8 ,3 ,2 ,1 ,6 ,8 ,2 ,4 ,0 ,7 ,5 ,5 ,7 ,7 ,6 ,9 ,0 ,3 ,6 ,7 ,3 ,9 ,4 ,4 ,2 ,0 ,7 ,5 ,8 ,1 ,2 ,4 ,4 ,3 ,2 ,4 ,6 ,7 ,6 ,7 ,5 ,9 ,2 ,0 ,1 ,0 ,4/2 ,7 ,0 ,0 ,2 ,7 ,9 ,3 ,3 ,9 ,2 ,3 ,6 ,0 ,8 ,0 ,2 ,3 ,4 ,2 ,6 ,4 ,4 ,1 ,1 ,5 ,0 ,0 ,6 ,0 ,9 ,3 ,5 ,0 ,4 ,3 ,9 ,8 ,6 ,8 ,3 ,8 ,1 ,2 ,5 ,2 ,8 ,9 ,4 ,0'
# x = '0 ,8 ,7 ,5 ,0 ,3 ,6 ,5 ,7 ,8/8 ,0 ,8 ,2 ,7 ,2 ,7 ,3 ,2 ,9/7 ,8 ,0 ,5 ,3 ,9 ,6 ,4 ,7 ,3/5 ,2 ,5 ,0 ,4 ,8 ,4 ,7 ,1 ,8/0 ,7 ,3 ,4 ,0 ,2 ,4 ,2 ,8 ,1/3 ,2 ,9 ,8 ,2 ,0 ,8 ,4 ,7 ,7/6 ,7 ,6 ,4 ,4 ,8 ,0 ,5 ,9 ,7/5 ,3 ,4 ,7 ,2 ,4 ,5 ,0 ,5 ,9/7 ,2 ,7 ,1 ,8 ,7 ,9 ,5 ,0 ,9/8 ,9 ,3 ,8 ,1 ,7 ,7 ,9 ,9 ,0'
# x = '0 ,2 ,5 ,2 ,1 ,3 ,1 ,0 ,8 ,4 ,2 ,9 ,2 ,3 ,6 ,2 ,7 ,0 ,4 ,9/2 ,0 ,1 ,7 ,1 ,1 ,3 ,1 ,8 ,3 ,5 ,5 ,6 ,2 ,6 ,5 ,6 ,6 ,6 ,9/5 ,1 ,0 ,9 ,5 ,9 ,6 ,7 ,7 ,0 ,4 ,2 ,4 ,3 ,9 ,4 ,2 ,5 ,5 ,4/2 ,7 ,9 ,0 ,7 ,3 ,3 ,5 ,3 ,9 ,1 ,5 ,1 ,0 ,8 ,3 ,9 ,8 ,1 ,5/1 ,1 ,5 ,7 ,0 ,1 ,1 ,2 ,7 ,3 ,6 ,5 ,7 ,9 ,9 ,8 ,0 ,6 ,4 ,3/3 ,1 ,9 ,3 ,1 ,0 ,8 ,7 ,2 ,1 ,6 ,7 ,8 ,8 ,1 ,5 ,7 ,6 ,8 ,7/1 ,3 ,6 ,3 ,1 ,8 ,0 ,1 ,3 ,2 ,1 ,4 ,1 ,4 ,9 ,8 ,1 ,8 ,5 ,8/0 ,1 ,7 ,5 ,2 ,7 ,1 ,0 ,1 ,6 ,4 ,0 ,3 ,1 ,8 ,5 ,4 ,5 ,9 ,6/8 ,8 ,7 ,3 ,7 ,2 ,3 ,1 ,0 ,6 ,5 ,7 ,7 ,0 ,6 ,1 ,8 ,7 ,5 ,5/4 ,3 ,0 ,9 ,3 ,1 ,2 ,6 ,6 ,0 ,7 ,3 ,0 ,2 ,1 ,5 ,2 ,9 ,6 ,7/2 ,5 ,4 ,1 ,6 ,6 ,1 ,4 ,5 ,7 ,0 ,0 ,5 ,5 ,3 ,5 ,6 ,7 ,9 ,2/9 ,5 ,2 ,5 ,5 ,7 ,4 ,0 ,7 ,3 ,0 ,0 ,1 ,1 ,5 ,9 ,1 ,3 ,8 ,5/2 ,6 ,4 ,1 ,7 ,8 ,1 ,3 ,7 ,0 ,5 ,1 ,0 ,7 ,1 ,6 ,8 ,7 ,4 ,6/3 ,2 ,3 ,0 ,9 ,8 ,4 ,1 ,0 ,2 ,5 ,1 ,7 ,0 ,6 ,3 ,6 ,3 ,2 ,2/6 ,6 ,9 ,8 ,9 ,1 ,9 ,8 ,6 ,1 ,3 ,5 ,1 ,6 ,0 ,5 ,0 ,2 ,3 ,3/2 ,5 ,4 ,3 ,8 ,5 ,8 ,5 ,1 ,5 ,5 ,9 ,6 ,3 ,5 ,0 ,1 ,7 ,7 ,0/7 ,6 ,2 ,9 ,0 ,7 ,1 ,4 ,8 ,2 ,6 ,1 ,8 ,6 ,0 ,1 ,0 ,0 ,4 ,6/0 ,6 ,5 ,8 ,6 ,6 ,8 ,5 ,7 ,9 ,7 ,3 ,7 ,3 ,2 ,7 ,0 ,0 ,4 ,9/4 ,6 ,5 ,1 ,4 ,8 ,5 ,9 ,5 ,6 ,9 ,8 ,4 ,2 ,3 ,7 ,4 ,4 ,0 ,5/9 ,9 ,4 ,5 ,3 ,7 ,8 ,6 ,5 ,7 ,2 ,5 ,6 ,2 ,3 ,0 ,6 ,9 ,5 ,0'
# x = x.split('/')
# g = [[] for _ in range(len(x))]
# for i in range(len(x)):
#   x[i]=x[i].split(',')
#   for j in range(len(x)):
#     g[i].append(int(x[i][j].strip()))
# graph = g
# graph = [[0,4,1,3],
#          [4,0,2,1],
#          [1,2,0,5],
#          [3,1,5,0]]
# Zurich, Warsaw, Vienna, Bucharest, Amsterdam, Venice, Stuttgart, Geneva
graph = [[0, 1331, 721, 1797, 813, 540, 217, 277],
          [1331, 0, 671, 1389, 1193, 1271, 1120, 1598],
          [721, 671, 0, 1071, 1145, 609, 622, 995],
          [1797, 1389, 1071, 0, 2228, 1528, 1699, 2092],
          [813, 1193, 1145, 2228, 0, 1334, 615, 982],
          [540, 1271, 609, 1528, 1334, 0, 698, 578 ],
          [217, 1120, 622, 1699, 615, 698, 0, 491],
          [277, 1598, 995, 2092, 982, 578,491, 0] ]
print("Solving the graph: ")
tsp = tour(graph)
solStatus = successorFn(tsp)
print(solStatus)

Solving the graph: 
At Level:  0 , go to:  Zurich . Total cost of tour till now:  0
At Level:  1 , go to:  Stuttgart . Total cost of tour till now:  217
At Level:  2 , go to:  Amsterdam . Total cost of tour till now:  832
At Level:  3 , go to:  Geneva . Total cost of tour till now:  1814
At Level:  4 , go to:  Venice . Total cost of tour till now:  2392
At Level:  5 , go to:  Vienna . Total cost of tour till now:  3001
At Level:  6 , go to:  Warsaw . Total cost of tour till now:  3672
At Level:  7 , go to:  Bucharest . Total cost of tour till now:  5061
Going back to Hometown Zurich from the last stop  Bucharest  costs:  1797
Total Cost of Tour:  6858
Solved
